# Why Are Our Customers Churning?

### Quick Reference
I. Find Baseline Probability  
II. Create a Baseline Model  
III. Create/Test Models and Identify Ω Model  

## I. Project Plan

### Summary

Zach, my team leader @ Telco Corp, wants to find out why our customers churning.  

Below is a list of questions he wants answers to: _tk, theres a new question from Maggie_

1. Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
2. Is there a price threshold for specific services where the likelihood of churn increases once price for those services goes past that point? If so, what is that point for what service(s)?
3. If we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?

### Goals

The goals of the project are to answer the questions above and to deliver the following data products:

1. Report detailing my analysis in an .ipynb format
2. CSV file containing my predictions on a test data set
3. Google Slides explaining my chosen model
4. .py files that are used through the entire pipeline, and that contains _reproducible_ python scripts
5. Read Me file on a github repo containing all files created for this project
  
### Data Dictionary
| Feature 	| Description 	| Table of Origin 	| Notes 	|
|---------	|-------------	|-----------------	|-------	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|
|         	|             	|                 	|       	|

### Packages

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import pandas_profiling
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

import wrangle
import prep
import model
import explore

from sklearn.model_selection import train_test_split
from sklearn.tree import export_graphviz
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

from scipy.stats import binom

## I. Find Baseline Probability

With the available data, what is the proportion of customers who churn?

```sql
SELECT churn, count(*) /
	(SELECT count(*) FROM customers)
FROM customers
WHERE churn = "Yes"
```

Proportion of people who churned:
0.2654 (out of 7043 observations)

n = no. of trials  
P = probability of success (success = Churned)

In [2]:
will_churn = binom(7043, 0.2654).sf(.5) # not sure about this yet. will come back later tk
will_churn

1.0

## II. Acquire and Split Data

Using the `get_sql_telcochurn` function from `wrangle.py`, acquire data from `telco_churn` database on MySQL.

In [3]:
telco = wrangle.get_sql_telco()

Split data to train and test, and set aside test data.

In [4]:
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customer_id                 7043 non-null object
gender                      7043 non-null object
senior_citizen              7043 non-null int64
partner                     7043 non-null object
dependents                  7043 non-null object
tenure                      7043 non-null int64
phone_service               7043 non-null object
multiple_lines              7043 non-null object
internet_service_type_id    7043 non-null int64
online_security             7043 non-null object
online_backup               7043 non-null object
device_protection           7043 non-null object
tech_support                7043 non-null object
streaming_tv                7043 non-null object
streaming_movies            7043 non-null object
contract_type_id            7043 non-null int64
paperless_billing           7043 non-null object
payment_type_id             7043 non-null int64
monthly_charges 

In [5]:
telco = telco.replace(" ",np.nan)
telco.isnull().sum()

customer_id                  0
gender                       0
senior_citizen               0
partner                      0
dependents                   0
tenure                       0
phone_service                0
multiple_lines               0
internet_service_type_id     0
online_security              0
online_backup                0
device_protection            0
tech_support                 0
streaming_tv                 0
streaming_movies             0
contract_type_id             0
paperless_billing            0
payment_type_id              0
monthly_charges              0
total_charges               11
churn                        0
dtype: int64

`total_charges` is an object type. There are 11 observations in the dataset that are encoded as empty. These observations are making the `total_charges` attribute an object instead of a float type.

With the quantity of our total data set (7043) and the time alloted for the project, I could afford losing 11 observations to facilitate analysis. Therefore, I am dropping all the nulls hereon.

>Action Steps:  
 >- Drop NaNs using `dropna()`
 >- Cast `total_charges` values to a float data type

In [6]:
telco = telco.dropna()
telco.total_charges = telco.total_charges.astype("float")

print(f"""
From the original train count of 7043, telco now has a {len(telco)} observations. Also, total charges is now a "{telco.total_charges.dtype}" data type.
""")


From the original train count of 7043, telco now has a 7032 observations. Also, total charges is now a "float64" data type.



### Split Test and Train Data

In [7]:
train, test = train_test_split(telco, train_size=0.7, random_state=123)

print(f"""
Train data size: {len(train)}
Test data size: {len(test)}
""")


Train data size: 4922
Test data size: 2110



## III. Perform Temporary Cleaning and Preliminary Exploration on Train Data

In [8]:
telco.total_charges.value_counts(ascending=False, dropna=False)

20.20      11
19.75       9
20.05       8
19.65       8
19.90       8
45.30       7
19.55       7
20.25       6
19.45       6
20.15       6
20.30       5
20.45       5
49.90       4
69.60       4
19.40       4
69.95       4
19.50       4
70.60       4
20.35       4
20.50       4
19.85       4
69.90       4
19.30       4
19.20       4
44.40       4
69.65       4
50.15       4
19.95       4
75.30       4
44.00       4
           ..
6981.35     1
890.60      1
1414.80     1
4566.50     1
369.25      1
2658.80     1
3638.25     1
120.25      1
1534.05     1
7262.00     1
488.25      1
4483.95     1
3320.60     1
126.05      1
1374.35     1
7238.60     1
184.95      1
1982.60     1
1834.15     1
187.75      1
249.95      1
8333.95     1
7171.70     1
3726.15     1
3928.30     1
964.90      1
5958.85     1
414.10      1
68.75       1
30.50       1
Name: total_charges, Length: 6530, dtype: int64

## III. Create a Baseline Decision Tree Model

Because Decision Tree accepts discrete and continuous features, I will create baseline model using a Decision Tree.  
> Action Steps:
> - Encode `churn` into a computer-readable variable with 0s and 1s values, such that 0 = No (Stayed), 1 = Yes (Churned). Use `int_encode` function from `prep`
   

In [9]:
train["enc_churn"] = train.churn.apply(prep.yes_no_to_boolean)

In [10]:
X_train=train[["senior_citizen","tenure","internet_service_type_id","contract_type_id","payment_type_id","monthly_charges"]]
y_train=train.enc_churn

### Create a baseline model using Decision Tree

Using `model_by_cart` function from `model.py`, create a decision tree model using all variables with numeric data as features (X), and the encoded churn variable `train.enc_churn` as the target variable (y).

In [11]:
yhat_trainb, yhat_trainb_proba, dt = model.model_by_cart(X_train,y_train,"entropy",3)

In [12]:
print(f"""
Baseline Predictions:
{yhat_trainb}
  
Probabilities of Baseline Predictions:
{yhat_trainb_proba}
""")


Baseline Predictions:
[0 0 0 ... 0 0 0]
  
Probabilities of Baseline Predictions:
[[0.80952381 0.19047619]
 [0.55687204 0.44312796]
 [0.55687204 0.44312796]
 ...
 [0.60187668 0.39812332]
 [0.95333333 0.04666667]
 [0.80952381 0.19047619]]



In [13]:
trainb_accuracy = dt.score(X_train, y_train)
print(trainb_accuracy)

0.7870784234051199


In [14]:
labels = ["stayed","churned"]
trainb_conf = pd.DataFrame(confusion_matrix(y_train, yhat_trainb), index=labels, columns=labels)
trainb_conf

,stayed,churned
stayed,3343,267
churned,781,531


In [15]:
trainb_report = classification_report(y_train, yhat_trainb)
print(trainb_report)

              precision    recall  f1-score   support

           0       0.81      0.93      0.86      3610
           1       0.67      0.40      0.50      1312

    accuracy                           0.79      4922
   macro avg       0.74      0.67      0.68      4922
weighted avg       0.77      0.79      0.77      4922



![baseline_decision_tree](baseline_dtb.jpg)

This is the code to produce the above decision tree diagram.
>dot_data = export_graphviz(dt, out_file=None)  
>graph = graphviz.Source(dot_data)  
>graph.render('telco_dtb', view=True)

## V. Create/Test Models

<div class="alert alert-block alert-info">
<b>Question:</b> Are there features that indicate a higher propensity to churn? like type of internet service, type of phone service, online security and backup, senior citizens, paying more than x% of customers with the same services, etc.?
</div>

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4922 entries, 463 to 3591
Data columns (total 22 columns):
customer_id                 4922 non-null object
gender                      4922 non-null object
senior_citizen              4922 non-null int64
partner                     4922 non-null object
dependents                  4922 non-null object
tenure                      4922 non-null int64
phone_service               4922 non-null object
multiple_lines              4922 non-null object
internet_service_type_id    4922 non-null int64
online_security             4922 non-null object
online_backup               4922 non-null object
device_protection           4922 non-null object
tech_support                4922 non-null object
streaming_tv                4922 non-null object
streaming_movies            4922 non-null object
contract_type_id            4922 non-null int64
paperless_billing           4922 non-null object
payment_type_id             4922 non-null int64
monthly_charge

Creating a new attribute `enc_phone_service` that tells what phone service types customers have:
  - 0: No phone service
  - 1: Single line phone service
  - 2: Multiple lines phone service

In [38]:
enc= OrdinalEncoder(categories=[["No phone service","No","Yes"]],dtype=int)
enc = enc.fit(train[["multiple_lines"]])
train["enc_phone_service"] = enc.transform(train[["multiple_lines"]])

In [40]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,streaming_tv,streaming_movies,contract_type_id,paperless_billing,payment_type_id,monthly_charges,total_charges,churn,enc_churn,enc_phone_service
463,0678-RLHVP,Female,0,No,No,53,Yes,Yes,2,No,...,Yes,Yes,3,No,1,105.55,5682.25,No,0,2
5828,8203-XJZRC,Male,0,No,No,1,Yes,No,3,No internet service,...,No internet service,No internet service,1,No,2,20.25,20.25,No,0,1
1433,2111-DWYHN,Male,0,No,No,1,Yes,No,3,No internet service,...,No internet service,No internet service,1,No,2,21.10,21.10,No,0,1
2892,4119-ZYPZY,Male,1,No,No,71,Yes,Yes,2,No,...,Yes,Yes,3,Yes,1,106.80,7623.20,No,0,2
3915,5512-IDZEI,Male,0,Yes,Yes,46,Yes,Yes,3,No internet service,...,No internet service,No internet service,2,No,1,24.90,1174.80,No,0,2


Creating a new attribute `enc_household` that tells what phone service types customers have:
  - 0: No phone service
  - 1: Single line phone service
  - 2: Multiple lines phone service

In [ ]:
en

In [ ]:
features = train.iloc[:,1:20].columns.tolist()
target = "enc_churn"

explore.plot_bars(features, target, train)

**Questions I have after the simple vizes above:**

1. Senior citizens churn more, why?
    - high monthly charges? Why are they paying higher?
    - electronic checks?
    - online security and protection, backup
    - device protection?
    - customer support?
    - contract type?
    
2. Paperless billing is higher and higher churn
 - higher monthly charge
 
3. Payment Type ID
- mailed checks have lesser monthly charges. control for senior citizen and other services

4. Single line, lower charges but higher churn?? why?

_$H0$: There is no difference in the exam scores for those who studied with flashcards and those who didn't.  
$H1$: There is a difference in the exam scores for those who studied with flashcards and those who didn't._

<div class="alert alert-block alert-info">
<b>Question:</b> Is there a price threshold for specific services where the likelihood of churn increases when the price for those services goes past that point? If so, what is that point for what service(s)?
</div>

In [ ]:
train.head()

In [ ]:
train["monthly_bins"] = pd.cut(train.monthly_charges,4)

In [ ]:
plt.subplot(121)
sns.barplot(x="monthly_bins", y="enc_churn", data=train, hue="internet_service_type_id")
plt.subplot(122)
sns.barplot(x="monthly_bins", y="enc_churn", data=train, hue="phone_service")

In [ ]:
sns.barplot(x="monthly_bins", y="enc_churn", data=train, hue="phone_service")

In [ ]:
sns.barplot(x=train.contract_type_id, y="monthly_charges",data=train,hue="churn")
plt.legend()
plt.hlines(y=train.monthly_charges.mean(), xmin=-1, xmax=3, ls=":")

In [ ]:
explore.plot_box(features,target,train)
plt.legend()

In [ ]:
sns.barplot(x=train.contract_type_id, y="monthly_charges",data=train,hue="churn")
plt.legend()

<div class="alert alert-block alert-info">
<b>Question:</b> IIf we looked at churn rate for month-to-month customers after the 12th month and that of 1-year contract customers after the 12th month, are those rates comparable?
</div>

**Answer:**  
After the 12-month period lapsed, 38.3% churned. Out of those, 37% are on a month-to-month contract while only 0.1% are on a one-year contract.

In [ ]:
twelve_months = train[train.tenure == 13]
pd.crosstab(twelve_months.churn, twelve_months.contract_type_id,normalize=True)

In [ ]:
sns.barplot(y="enc_churn",x="contract_type_id", data=twelve_months, hue="contract_type_id")